# preparation

## imports

In [1]:
# environment imports
import sys, os, shutil, glob, random
import warnings
warnings.filterwarnings('ignore')

# visualization imports
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('classic')

import seaborn as sns
sns.set_style("whitegrid")

In [2]:
import numpy as np
import math, timm, time
import pandas as pd

# PyTorch import block
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim.lr_scheduler as lrs
from   torch.utils.data import Dataset, DataLoader, TensorDataset
from   torch.utils.data.dataloader import default_collate
from   torch.optim.optimizer import Optimizer
from   torch.nn import TransformerEncoderLayer


# Torchvision import block
import torchvision
import torchvision.transforms as transforms

# einpos import block
import einops
from   einops import rearrange, repeat
from   einops.layers.torch import Rearrange

from tqdm.notebook import tqdm
from scipy import stats

from typing import Optional
import copy

In [3]:
print(f'current packages version:\n* torch: {torch.__version__};' \
      + f'\n* timm: {timm.__version__};\n* torchvision: {torchvision.__version__}')

current packages version:
* torch: 2.2.1;
* timm: 0.6.12;
* torchvision: 0.17.1


## variables

In [4]:
DATA_DIR, LOCAL_DIR, FE_MODELS_DIR = './data', './', './models/feature_extractors'

TRAIN_MP4_DIR, TRAIN_WAV_DIR = f'{DATA_DIR}/train/mp4', f'{DATA_DIR}/train/wav'
VALID_MP4_DIR, VALID_WAV_DIR = f'{DATA_DIR}/val/mp4', f'{DATA_DIR}/val/wav'

TRAIN_MP4_FEATURES_DIR, TRAIN_WAV_FEATURES_DIR = f'{DATA_DIR}/train/mp4_features', f'{DATA_DIR}/train/wav_features'
VALID_MP4_FEATURES_DIR, VALID_WAV_FEATURES_DIR = f'{DATA_DIR}/val/mp4_features', f'{DATA_DIR}/val/wav_features'

In [5]:
ABAW5_MODELS_CHECKPOINTS = './models/abaw_checkpoints'

In [6]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Current connected device is {DEVICE}')

Current connected device is cuda


## classes

In [7]:
def sinusoidal_embedding(n_channels: int=None, dim: int=None):
    '''
    Function define Positional Encoding.
    
            Parameters:
                n_channels (int): Number of channels in the input;
                dim (int): Dimension value.
    '''
    pe = torch.FloatTensor([[p / (10000 ** (2 * (i // 2) / dim)) for i in range(dim)] for p in range(n_channels)])
    pe[:, 0::2] = torch.sin(pe[:, 0::2])
    pe[:, 1::2] = torch.cos(pe[:, 1::2])
    
    return rearrange(pe, '... -> 1 ...')

class PreNorm(nn.Module):
    """Class for layer normalization in the current network"""
    def __init__(self, dim: int=None, fn: nn.Sequential=None):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
        
    def forward(self, x: torch.Tensor=None, **kwargs):
        return self.fn(self.norm(x), **kwargs)
    
class FeedForward(nn.Module):
    """Feed-forward block for the Transformer class"""
    def __init__(self, dim: int=None, hidden_dim: int=None, dropout: float=0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
        
    def forward(self, x: torch.Tensor=None):
        return self.net(x)
    
class Attention(nn.Module):
    """Attention mechanism realiztion"""
    def __init__(self, dim: int=None, heads: int=8, dim_head: int=64, dropout: float=0.):
        super().__init__()
        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim = -1)
        self.dropout = nn.Dropout(dropout)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x: torch.Tensor=None):
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        
        return self.to_out(out)
    
class Transformer(nn.Module):
    """Transformer block class"""
    def __init__(
        self, dim: int=None, depth: int=None, heads: int=None, 
        dim_head: int=None, mlp_dim: int=None, dropout: float=0.
    ):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))
            
    def forward(self, x: torch.Tensor=None):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
            
        return x

In [8]:
def get_prob(
    features:np.ndarray=None, classifier_weights:np.ndarray=None, classifier_bias:np.ndarray=None, 
    logits:bool=True
) -> np.ndarray:
    '''
    Function for the getting probabilities of the classes of the feature_extraction model.

            Parameters:
                features (np.ndarray): Current extracted features;
                classifier_weights (np.ndarray): Classifier weights;
                classifier_bias(np.ndarray): Classifier bias;
                logits (bool): Get the logits or not.
    '''
    xs = np.dot(features, np.transpose(classifier_weights)) + classifier_bias

    if logits:
        return xs
    else:
        e_x = np.exp(xs - np.max(xs, axis=1)[:,np.newaxis])
        return e_x / e_x.sum(axis=1)[:, None]

def stack_npy_files(directory: str=None):
    """
    Open all .npy files in the given directory and stack them into one big array.
    
            Parameters:
                directory (str): The directory containing the .npy files.
    """
    npy_files = [file for file in os.listdir(directory) if file.endswith('.npy')]
    if not npy_files:
        print("No .npy files found in the directory.")
        return None
    
    stacked_array = np.concatenate([np.load(os.path.join(directory, file)) for file in npy_files], axis=0)
    return stacked_array

class VisualModalityDataset(Dataset):
    """Dataset class for the visual-modality"""
    
    def __init__(
        self, data_info_path: str=None, split: str='Train', visual_features_dir_path: str=None,
        feature_extractor_head: timm.models.EfficientNet=None, use_scores: bool=True,
        max_seq_length: int=64
    ):
        super(VisualModalityDataset, self).__init__()
        self.data_info = pd.read_csv(data_info_path)
        self.split, self.visual_features_dir_path = split, visual_features_dir_path
        self.data_info = self.data_info[self.data_info['Split'] == self.split]
        self.file_name_padding, self.target_labels = 5, [
            'Adoration', 'Amusement', 'Anxiety', 'Disgust','Empathic-Pain', 'Fear', 'Surprise'
        ]
        
        self.use_scores, self.max_seq_length = use_scores, max_seq_length
        self.visual_weights = feature_extractor_head.classifier.weight.cpu().data.numpy()
        self.visual_bias = feature_extractor_head.classifier.bias.cpu().data.numpy()
        
        self.__init_inputs_labels()
        
    def __init_inputs_labels(self):
        print(f'Configure dataset from directory {self.visual_features_dir_path}')
        self.inputs, self.meta, self.labels = [], [], []
        
        for index, row_values in tqdm(self.data_info.iterrows(), total=self.data_info.shape[0]):
            current_ID = str(row_values['ID'])
            current_file_ID = '0' * (self.file_name_padding - len(current_ID)) + current_ID
            
            feature_dir_path = f'{self.visual_features_dir_path}/{current_file_ID}_batched_features'
            current_visual_features = stack_npy_files(directory=feature_dir_path)
            if current_visual_features is None:
                continue
            seq_length = current_visual_features.shape[0]
            
            if seq_length > self.max_seq_length: 
                current_visual_features = current_visual_features[:self.max_seq_length]
            else:
                current_visual_features = np.pad(
                    current_visual_features, pad_width=((0, self.max_seq_length - seq_length),(0,0)))
            
            if self.use_scores:
                scores = []
                for row in range(current_visual_features.shape[0]):
                    row_visual_feature = current_visual_features[row, :]
                    current_scores = get_prob(row_visual_feature, self.visual_weights, self.visual_bias)
                    scores.append(current_scores)

                scores = np.stack(scores, axis=0)
                current_visual_features = np.concatenate((current_visual_features, scores), axis=1)
            
            current_labels = row_values[self.target_labels].values
            current_meta_info = [row_values['Age'], int(row_values['Country'] in ['United States'])]
            
            self.inputs.append(current_visual_features)
            self.labels.append(current_labels)
            self.meta.append(current_meta_info)
            
            torch.cuda.empty_cache()
            
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index: int=None):
        inputs, labels, meta_info = self.inputs[index], self.labels[index], self.meta[index]
        
        inputs = torch.tensor(inputs.astype(np.float32), dtype=torch.float)
        labels = torch.tensor(labels.astype(np.float32), dtype=torch.float)
        meta_info = torch.tensor(np.array(meta_info).astype(np.float32), dtype=torch.float)
        
        return inputs, labels, meta_info

## functions

In [9]:
def seed_everything(seed:int=None) -> None:
    '''
    Function seed every random asprect.

            Parameters:
                seed (int): The seed number.
    '''
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    
def configure_feature_extraction_model_visual(
    feature_extractor_model_path: str=None, device: torch.device=None, return_initial: bool=True
) -> np.ndarray:
    '''
    Function configure feature-extraction model
    
            Parameters:
                extraction_model_path (str): path to feature-extraction model;
                device (torch.device): torch device (default=torch.cuda);
                return_initial (bool): Return the initial model or not
            Returns:
                features-extraction model
    '''
    feature_extractor_model = torch.load(feature_extractor_model_path)
    feature_extractor_model.classifier = torch.nn.Identity()
    feature_extractor_model.to(device)
    feature_extractor_model.eval()

    if return_initial:
        return feature_extractor_model, torch.load(feature_extractor_model_path)
    else:
        return feature_extractor_model
    
def calc_pearsons(predictions:np.array=None, ground_truth:np.array=None):
    '''
    Function calculates Pearson's Correlation Coefficient.
    
            Parameters:
                predictions (np.array): Model's forecasts;
                ground_truth (np.array): The fact.
    '''
    pcc = stats.pearsonr(predictions, ground_truth)
    return pcc[0]

def mean_pearsons(predictions:np.array=None, ground_truth:np.array=None, n_classes:int=7):
    '''
    Function calculates mean PCC between predictions and fact.
    
            Parameters:
                predictions (np.array): Model's forecasts;
                ground_truth (np.array): The fact;
                n_classes (int): number of classes.
    '''
    predictions, ground_truth = predictions.detach().cpu().numpy(), ground_truth.detach().cpu().numpy()
    class_wise_pcc = np.array([calc_pearsons(predictions[:, i], ground_truth[:, i]) for i in range(n_classes)])
    mean_pcc = np.mean(class_wise_pcc)
    
    return mean_pcc, class_wise_pcc

In [26]:
def train_one_epoch_visual(
    model:nn.Module=None, train_dataloader:DataLoader=None, criterion:nn.Module=None, 
    optimizer:torch.optim.Optimizer=None, scheduler:torch.optim.lr_scheduler=None, device=None, mode:str=None,
    use_meta: bool=False
):
    '''
    Function perform one epoch train iteration.
    
            Parameters:
                model (nn.Module): Current model;
                train_dataloder (Dataloader): Current train dataloader;
                criterion (nn.Module): Loss-function;
                optimizer (torch.optim.Optimizer): Current optimization function;
                scheduler (torch.optim.Scheduler): Current scheduler;
                device: Current device;
                mode (str): Current mode ('only-visual', 'only-audio', 'multi-modal')
    '''
    train_loss, train_mean_pcc, total_outputs, total_labels = 0, 0, [], []
    model.train()
    
    for current_batch in tqdm(train_dataloader):
        visual_features, labels, meta_info = current_batch
        visual_features, labels, meta_info = torch.tensor(data=visual_features, dtype=torch.float32), \
            torch.tensor(data=labels, dtype=torch.float32), torch.tensor(data=meta_info, dtype=torch.float32)
        visual_features, labels, meta_info = visual_features.to(device), labels.to(device), meta_info.to(device)
        
        if use_meta: visual_features = torch.concat((visual_features, meta_info.unsqueeze(1).expand(-1, 128, -1)), dim=-1)
        
        optimizer.zero_grad()
        
        outputs = model(visual_features)
        
        train_batch_loss = criterion(labels, outputs)
        train_batch_loss.backward()
        train_loss += train_batch_loss.item()
        
        batch_mean_pcc, _ = mean_pearsons(outputs, labels)
        train_mean_pcc += batch_mean_pcc
        
        optimizer.step()
            
    train_loss = train_loss / len(train_dataloader)
    train_mean_pcc = train_mean_pcc / len(train_dataloader)
    
    return train_loss, train_mean_pcc

In [27]:
def eval_one_epoch_visual(
    model:nn.Module=None, valid_dataloader:DataLoader=None, criterion:nn.Module=None, 
    scheduler:torch.optim.lr_scheduler=None, device=None, mode:str=None,
    use_meta: bool=False, model_checkpoint_dir: str=None, best_validation_score: float=None,
    model_suffix: str=None
):
    '''
    Function perform one epoch train iteration.
    
            Parameters:
                model (nn.Module): Current model;
                train_dataloder (Dataloader): Current train dataloader;
                criterion (nn.Module): Loss-function;
                optimizer (torch.optim.Optimizer): Current optimization function;
                scheduler (torch.optim.Scheduler): Current scheduler;
                device: Current device;
                mode (str): Current mode ('only-visual', 'only-audio', 'multi-modal')
    '''
    valid_loss, valid_mean_pcc, total_outputs, total_labels = 0, 0, [], []
    model.eval()
    
    with torch.no_grad():
        for current_batch in tqdm(valid_dataloader):
            visual_features, labels, meta_info = current_batch
            visual_features, labels, meta_info = torch.tensor(data=visual_features, dtype=torch.float32), \
                torch.tensor(data=labels, dtype=torch.float32), torch.tensor(data=meta_info, dtype=torch.float32)
            visual_features, labels, meta_info = visual_features.to(device), labels.to(device), meta_info.to(device)

            if use_meta: visual_features = torch.concat(
                (visual_features, meta_info.unsqueeze(1).expand(-1, 128, -1)), dim=-1)

            outputs = model(visual_features)

            valid_batch_loss = criterion(labels, outputs)
            valid_loss += valid_batch_loss.item()

            batch_mean_pcc, _ = mean_pearsons(outputs, labels)
            valid_mean_pcc += batch_mean_pcc

        valid_loss = valid_loss / len(valid_dataloader)
        valid_mean_pcc = valid_mean_pcc / len(valid_dataloader)
        
        if scheduler is not None:
            scheduler.step()

    if valid_mean_pcc >= best_validation_score:
        torch.save(model.state_dict(), f'{model_checkpoint_dir}/{model_suffix}_{valid_mean_pcc:.4f}.pt')
        
    return valid_loss, valid_mean_pcc

In [28]:
def calculate_apcc(preds, labels):
    preds_mean = torch.mean(preds, dim=0, keepdim=True)
    labels_mean = torch.mean(labels, dim=0, keepdim=True)

    pcc = torch.sum((preds-preds_mean) * (labels-labels_mean), dim=0) / \
        torch.clamp((torch.sum((preds-preds_mean)**2, dim=0) * torch.sum((labels-labels_mean)**2, dim=0))**0.5, min=1e-8)

    return torch.mean(pcc)

def hard_sample_mask(losses, flag=True):
    mask = torch.ones(losses.shape).to(losses.device)
    if not flag:
        return mask

    # hard samples
    quantile = torch.quantile(losses, 0.75, interpolation='nearest')
    mask[losses > quantile] = 2.

    return mask

def compute_loss(preds, labels):
    loss_l2 = F.mse_loss(preds, labels, reduction='none')
    mask = hard_sample_mask(loss_l2, False)
    loss = torch.mean(loss_l2 * mask)
    
    return loss

# Model

## dataset

In [29]:
seed_everything(2002)
_, feature_extractor_model = configure_feature_extraction_model_visual(
    feature_extractor_model_path=f'{FE_MODELS_DIR}/efficientnet_affectnet.pt', device=DEVICE, return_initial=True)

In [30]:
start_time_extraction = time.time()

train_dataset = VisualModalityDataset(
    data_info_path=f'{DATA_DIR}/preprocessed_data_info.csv', split='Train', 
    visual_features_dir_path=TRAIN_MP4_FEATURES_DIR + '-HSEmotion',
    feature_extractor_head=feature_extractor_model
)
valid_dataset = VisualModalityDataset(
    data_info_path=f'{DATA_DIR}/preprocessed_data_info.csv', split='Val', 
    visual_features_dir_path=VALID_MP4_FEATURES_DIR + '-HSEmotion',
    feature_extractor_head=feature_extractor_model
)

elapsed_time = time.time() - start_time_extraction
print(f'Dataset were configured in {np.round((elapsed_time/60), 0)} minutes')

Configure dataset from directory ./data/train/mp4_features-HSEmotion


  0%|          | 0/15806 [00:00<?, ?it/s]

No .npy files found in the directory.
No .npy files found in the directory.
No .npy files found in the directory.
Configure dataset from directory ./data/val/mp4_features-HSEmotion


  0%|          | 0/4657 [00:00<?, ?it/s]

No .npy files found in the directory.
No .npy files found in the directory.
Dataset were configured in 5.0 minutes


## dataloader

In [31]:
class Collator(object):
    """Class for appropriate collate_fn"""
    def __init__(self, imgRandomLen: int=64):
        super().__init__()
        self.imgRandomLen = imgRandomLen

    def __call__(self, batch: torch.Tensor=None):
        '''
        Select a specfic number of images randomly for the time being.
        
                Parameters:
        '''
        images = np.stack([tensors[0] for tensors in batch], axis=0).astype(np.float)
        labels = np.stack([tensors[1] for tensors in batch], axis=0).astype(np.float)
        meta   = np.stack([tensors[2] for tensors in batch], axis=0).astype(np.float)
        
        return np.stack(images, axis=0).astype(np.float), labels, meta

In [43]:
collate_fn = Collator()
train_dataloader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True, collate_fn=collate_fn, drop_last=True)
valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=256, shuffle=False, collate_fn=collate_fn, drop_last=True)

## model

In [1]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
from torch.nn.init import xavier_uniform_, constant_

class Chomp1d(nn.Module):
    def __init__(self, chomp_size: int=None):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x: torch.Tensor=None):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(
        self, n_inputs: int=None, n_outputs: int=None, kernel_size: int=None, 
        stride: int=None, dilation: int=None, padding: int=None, dropout: float=0.2
    ):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x: torch.Tensor=None):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs: int=None, num_channels: int=None, kernel_size: int=2, dropout: float=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_channels = num_channels
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [
                TemporalBlock(
                    in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                    padding=(kernel_size-1) * dilation_size, dropout=dropout
                )]

        self.network = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor=None):
        return self.network(x)
    
from torch.autograd import Variable
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int=None, dropout: int=None, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        
        v = torch.arange(0, d_model, 2).type(torch.float)
        v = v * -(math.log(1000.0) / d_model)
        div_term = torch.exp(v)

        pe[:, 0::2] = torch.sin(position.type(torch.float) * div_term)
        pe[:, 1::2] = torch.cos(position.type(torch.float) * div_term)
        pe = pe.unsqueeze(0).to(DEVICE)

        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor=None):
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)
    
def clones(module: nn.Module=None, N: int=None):
    """
    Function for creating PyTorch modules clones.
    
            Parameters:
                module (nn.Module): Current PyTorch module;
                N (int): Number of clones.
    """
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()

        self.a_2 = nn.Parameter(torch.ones(features)).to(DEVICE)
        self.b_2 = nn.Parameter(torch.zeros(features)).to(DEVICE)
        self.eps = eps

    def forward(self, x: torch.Tensor=None):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()

        self.layers = layer
        self.norm = LayerNorm(layer[0].size)

    def forward(self, x: torch.Tensor=None, mask: torch.Tensor=None):
        for layer in self.layers:
            x = layer(x, mask)

        return self.norm(x)
    
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()

        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor=None, sublayer=None):
        return x + self.dropout(sublayer(self.norm(x)))
    
class EncoderLayer(nn.Module):
    def __init__(self, size: int=None, self_attn=None, feed_forward: int=None, dropout: float=0.2):
        super(EncoderLayer, self).__init__()

        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = nn.ModuleList()
        self.sublayer.append(SublayerConnection(size, dropout))
        self.sublayer.append(SublayerConnection(size, dropout))

        self.size = size

    def forward(self, x: torch.Tensor=None, mask: torch.Tensor=None):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)
    
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model: int=None, d_ff: int=None, dropout: float=0.1):
        super(PositionwiseFeedForward, self).__init__()

        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor=None):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))
    
def attention(
    query: torch.Tensor=None, key: torch.Tensor=None, value: torch.Tensor=None, 
    mask: torch.Tensor=None, dropout: float=None
):
    """
    Function for calculating attention mechanism.
    
            Parameters:
                query (torch.Tensor): Query;
                key (torch.Tensor): Key;
                value (torch.Tensor): Value;
    """
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)

    p_attn = F.softmax(scores, dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)

    return torch.matmul(p_attn, value), p_attn
    
class MultiHeadedAttention(nn.Module):
    def __init__(self, h: int=None, d_model: int=None, dropout: float=0.1):
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0

        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.dropout = nn.Dropout(p=dropout)

    def forward(
        self, query: torch.Tensor=None, key: torch.Tensor=None, value: torch.Tensor=None, mask: torch.Tensor=None
    ):
        if mask is not None:
            mask = mask.unsqueeze(1)

        nbatches = query.size(0)
        query, key, value = [
            l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2) \
            for l, x in zip(self.linears, (query, key, value))
        ]

        x, _ = attention(query, key, value, mask=mask, dropout=self.dropout)
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)

        return self.linears[-1](x)

In [46]:
class CustomTransformerModel(nn.Module):
    def __init__(self, input_size, d_model, n_heads, n_layers, n_classes, max_seq_length: int=64):
        super(CustomTransformerModel, self).__init__()
        self.tcn = TemporalConvNet(num_inputs=max_seq_length, num_channels=[max_seq_length, max_seq_length])
        self.pe = PositionalEncoding(d_model=input_size, dropout=0.3, max_len=5000)
        self.dropout_embed = nn.Dropout(p=0.3)
        
        self.projection_layer = nn.Linear(in_features=input_size, out_features=d_model)
        
        self.regress = nn.Sequential(
            nn.Linear(in_features=d_model * max_seq_length, out_features=d_model * max_seq_length // 2),
            nn.ReLU(),
            nn.Linear(in_features=d_model * max_seq_length // 2, out_features=n_classes)
        )
        
        encoder_layer = nn.ModuleList()
        for i in range(2):
            attention = MultiHeadedAttention(h=n_heads, d_model=d_model)
            ff = PositionwiseFeedForward(d_model=d_model, d_ff=d_model // 2)
            encoder_layer.append(EncoderLayer(size=d_model, self_attn=attention, feed_forward=ff))
        self.te = Encoder(encoder_layer, 2)
        
        for p in self.te.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
        for p in self.regress.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
        
    def forward(self, x: torch.Tensor=None):
        preprocessed_input = self.dropout_embed(self.pe(self.tcn(x)))
        preprocessed_input = self.te(self.projection_layer(preprocessed_input), mask=None)
        
        batch_size, seq_length, n_features = preprocessed_input.shape
        preprocessed_input = preprocessed_input.reshape(batch_size, -1)
        
        output = self.regress(preprocessed_input)
        return output

In [47]:
input_size, d_model = 1290, 128
n_heads, n_layers = 8, 6
n_classes = 7

model = CustomTransformerModel(input_size, d_model, n_heads, n_layers, n_classes).to(DEVICE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

In [51]:
n_epochs, model_suffix = 550, 'visual'
all_checkpoints = [
    float(checkpoint.split('_')[-1].replace('.pt', '')) for checkpoint 
    in os.listdir(ABAW5_MODELS_CHECKPOINTS) if ((checkpoint.endswith('.pt')) & (model_suffix in checkpoint))
]
best_val_pcc = max(all_checkpoints)

for epoch in range(n_epochs):
    if epoch in range(9):
        print(f"Epoch 0{epoch+1}/{n_epochs}")
    else:
        print(f"Epoch {epoch+1}/{n_epochs}")
        
    train_loss, train_mean_pcc = train_one_epoch_visual(
        model=model, train_dataloader=train_dataloader, criterion=criterion, optimizer=optimizer, 
        device=DEVICE, use_meta=False
    )
    print(f'training results. mean_pcc: {train_mean_pcc:.4f}; loss: {train_loss:.4f}')
    
    valid_loss, valid_mean_pcc = eval_one_epoch_visual(
        model=model, valid_dataloader=valid_dataloader, device=DEVICE, use_meta=False, criterion=criterion,
        scheduler=scheduler, model_checkpoint_dir=ABAW5_MODELS_CHECKPOINTS, best_validation_score=best_val_pcc,
        model_suffix=model_suffix
    )
    print(f'validating results. mean_pcc: {valid_mean_pcc:.4f}; loss: {valid_loss:.4f}')    

Epoch 01/550


  0%|          | 0/61 [00:00<?, ?it/s]

training results. mean_pcc: 0.3850; loss: 0.0930


  0%|          | 0/18 [00:00<?, ?it/s]

validating results. mean_pcc: 0.3648; loss: 0.0972
Epoch 02/550


...

validating results. mean_pcc: 0.3648; loss: 0.0972
Epoch 550/550


  0%|          | 0/61 [00:00<?, ?it/s]

training results. mean_pcc: 0.3850; loss: 0.0930


  0%|          | 0/18 [00:00<?, ?it/s]

validating results. mean_pcc: 0.3648; loss: 0.0972
